In [13]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import MatrixBasedConvolution as mbc

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Convolution parameters

In [14]:
inputs_shape_1d=(1,20)
inputs_shape_2d=(1,5,5,1)
ARGS={"kernel_size":3,
"stride":1,
"padding":'same',
"use_phi":True,
"activation":None,
"use_lambda_out":False,
"use_lambda_in":False}

## Convalution 1D and 2D outputs

In [15]:
print("1D Convolution building...")
con1d=mbc.matrix_conv_1d(**ARGS)
con1d.build(inputs_shape_1d)
print("2D Convolution building...")
con2d=mbc.matrix_conv_2d(**ARGS)
con2d.build(inputs_shape_2d)

1D Convolution building...
2D Convolution building...


In [16]:
print("inputs 1D generation...")
inputs_1d=tf.random.uniform(shape=inputs_shape_1d,minval=0,maxval=1)
print("inputs 2D generation...")
inputs_2d=tf.random.uniform(shape=inputs_shape_2d,minval=0,maxval=1)


inputs 1D generation...
inputs 2D generation...


In [17]:
ouputs_mbc_1d=con1d.conv_jit(inputs_1d).numpy()
print(ouputs_mbc_1d.shape)
ouputs_mbc_2d=con2d.conv_jit(inputs_2d).numpy()[0,:,:,0]
print(ouputs_mbc_2d.shape)

(1, 20)
(5, 5)


## Comparison between standard conv1d/conv2d and matrix based conv1d/conv2d

In [18]:
kernel=tf.reshape(con2d.kernel,shape=(ARGS.get('kernel_size'),ARGS.get('kernel_size')))
kernel.shape

TensorShape([3, 3])

In [19]:
ouputs_classical=mbc.classical_convolution(img=inputs_2d[0,:,:,0],kernel=kernel.numpy())
ouputs_classical.shape


(5, 5)

In [20]:
ouputs_mbc_2d

array([[-0.155317  , -0.21511647, -0.12905532, -0.6630691 , -0.19584489],
       [ 0.06053798, -0.30139107,  1.0559726 ,  0.02443737, -0.08383453],
       [ 0.36394474, -0.03728768, -0.20574373,  0.10574064, -0.36265993],
       [-0.26435986,  0.33647072, -0.2513414 ,  0.12431967, -0.13112715],
       [-0.34822625, -0.31560877,  0.09860386,  0.29490238, -0.19530796]],
      dtype=float32)

In [21]:
ouputs_classical

array([[-0.15531698, -0.21511647, -0.12905529, -0.6630691 , -0.1958449 ],
       [ 0.06053799, -0.30139107,  1.0559726 ,  0.02443737, -0.08383453],
       [ 0.36394477, -0.03728768, -0.20574379,  0.10574066, -0.36265993],
       [-0.26435983,  0.33647072, -0.2513414 ,  0.12431967, -0.13112713],
       [-0.34822625, -0.31560877,  0.09860385,  0.29490238, -0.19530796]],
      dtype=float32)

In [22]:
if np.allclose(ouputs_classical,ouputs_mbc_2d,atol=1e-16):
    print("The outputs are the same")
    

The outputs are the same
